## Import relevant modules, data and functions

In [2]:
import numpy as np
import rasterio as rs
import matplotlib.pyplot as plt
from numpy import loadtxt
from osgeo import gdal, gdalconst
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [3]:
#Plot Image
def read_data_EMP(inras):
    # Read data
    img = gdal.Open(inras, gdal.GA_ReadOnly) 
    bands = [img.GetRasterBand(i).ReadAsArray() for i in range(1, img.RasterCount + 1)]
    img = np.array(bands)
    img = img[0:12,:,:]
    img = np.transpose(img, [1, 2, 0])
    img_rgbnir = img[:,:,[3,2,1,7]]
    return img_rgbnir
def MyNormalize(img_i,sigma):
        nr,nc,nb = img_i.shape
        img_n = np.zeros(shape=(nr,nc,nb))
        for i in range(0,nb):
            one_band = img_i[:,:,i]
            mi = np.min(one_band)
            ma = np.max(one_band)
            one_band = (one_band-mi)/(ma-mi+np.finfo(float).eps)
            img_n[:,:,i] = sigma*one_band
        return img_n 

In [4]:
#Import Imagery
asi = 'Input/Imagery/SingleDateImage_Amsterdam.tif'
ami = 'Input/Imagery/MedianImage_Amsterdam.tif'
msi = 'Input/Imagery/SingleDateImage_Milano.tif'
mmi = 'Input/Imagery/MedianImage_Milano.tif'
bsi = 'Input/Imagery/SingleDateImage_Budapest.tif'
bmi = 'Input/Imagery/MedianImage_Budapest.tif'
#Import Feature extraction
EMPas = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empasi.npy')
NDWIas = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwiasi.csv', delimiter=',')
NDVIas = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndviasi.csv', delimiter=',')
EMPam = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empami.npy')
NDWIam = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwiami.csv', delimiter=',')
NDVIam = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndviami.csv', delimiter=',')
EMPms = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empmsi.npy')
NDWIms = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwimsi.csv', delimiter=',')
NDVIms = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndvimsi.csv', delimiter=',')
EMPmm = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empmmi.npy')
NDWImm = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwimmi.csv', delimiter=',')
NDVImm = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndvimmi.csv', delimiter=',')
EMPbs = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empbsi.npy')
NDWIbs = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwibsi.csv', delimiter=',')
NDVIbs = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndvibsi.csv', delimiter=',')
EMPbm = np.load('D:/Thesis/Notebooks/Output/Full/EMP/empbmi.npy')
NDWIbm = loadtxt('D:/Thesis/Notebooks/Output/Full/NDWI/ndwibmi.csv', delimiter=',')
NDVIbm = loadtxt('D:/Thesis/Notebooks/Output/Full/NDVI/ndvibmi.csv', delimiter=',')
#Import Twitter
Twita = rs.open('Input/Twitter/TwitterRaster_Amsterdam_fe.tif','r').read()
Twita = np.transpose(Twita, [1, 2, 0])
Twita = Twita[:,:,0]
Twitm = rs.open('Input/Twitter//TwitterRaster_Milano.tif','r').read()
Twitm = np.transpose(Twitm, [1, 2, 0])
Twitm = Twitm[:,:,0]
Twitb = rs.open('Input/Twitter/TwitterRaster_Budapest_fe.tif','r').read()
Twitb = np.transpose(Twitb, [1, 2, 0])
Twitb = Twitb[:,:,0]
#Import Validation datasets
Vala = rs.open('Input/Validation/Validation_30m_Amsterdam_fi.tif','r').read()
Vala = np.transpose(Vala, [1, 2, 0])
Val30ma = Vala[:,:,0]
Val30ma = np.where(Val30ma==2, 1, 0)
Val30ma = Val30ma.flatten()
Val2a = rs.open('Input/Validation/Validation_GAIA_Amsterdam_fi.tif','r').read()
Val2a = np.transpose(Val2a, [1, 2, 0])
Val2GAIAa = Val2a[:,:,0]
Val2GAIAa = np.where(Val2GAIAa>0, 1, 0)
Val2GAIAa = Val2GAIAa.flatten()
Val3a = rs.open('Input/Validation/Validation_GHS_Amsterdam_fi.tif', 'r').read()
Val3a = np.transpose(Val3a, [1, 2, 0])
Val3GHSa = Val3a[:,:,0]
Val3GHS50a = np.where(Val3GHSa>50, 1, 0)
Val3GHS50a = Val3GHS50a.flatten()

Valm = rs.open('Input/Validation/Validation_30m_Milano_fi.tif','r').read()
Valm = np.transpose(Valm, [1, 2, 0])
Val30mm = Valm[:,:,0]
Val30mm = np.where(Val30mm==2, 1, 0)
Val30mm = Val30mm.flatten()
Val2m = rs.open('Input/Validation/Validation_GAIA_Milano_fi.tif','r').read()
Val2m = np.transpose(Val2m, [1, 2, 0])
Val2GAIAm = Val2m[:,:,0]
Val2GAIAm = np.where(Val2GAIAm>0, 1, 0)
Val2GAIAm = Val2GAIAm.flatten()
Val3m = rs.open('Input/Validation/Validation_GHS_Milano_fi.tif', 'r').read()
Val3m = np.transpose(Val3m, [1, 2, 0])
Val3GHSm = Val3m[:,:,0]
Val3GHS50m = np.where(Val3GHSm>50, 1, 0)
Val3GHS50m = Val3GHS50m.flatten()

Valb = rs.open('Input/Validation/Validation_30m_Budapest_fi.tif','r').read()
Valb = np.transpose(Valb, [1, 2, 0])
Val30mb = Valb[:,:,0]
Val30mb = np.where(Val30mb==2, 1, 0)
Val30mb = Val30mb.flatten()
Val2b = rs.open('Input/Validation/Validation_GAIA_Budapest_fi.tif','r').read()
Val2b = np.transpose(Val2b, [1, 2, 0])
Val2GAIAb = Val2b[:,:,0]
Val2GAIAb = np.where(Val2GAIAb>0, 1, 0)
Val2GAIAb = Val2GAIAb.flatten()
Val3b = rs.open('Input/Validation/Validation_GHS_Budapest_fi.tif', 'r').read()
Val3b = np.transpose(Val3b, [1, 2, 0])
Val3GHSb = Val3b[:,:,0]
Val3GHS50b = np.where(Val3GHSb>50, 1, 0)
Val3GHS50b = Val3GHS50b.flatten()
from numpy import *
#GLCM_ASI_load = loadtxt('D:/Thesis/Notebooks/Output/Full/GLCM/ASI/GLCM_ASI.csv', delimiter=',')
#img = read_data_EMP(asi)
#nr, nc, nb = img.shape
#nf = 16
#GLCMas = GLCM_ASI_load.reshape(nr,nc,nf)
#where_are_NaNs = isnan(GLCMas)
#GLCMas[where_are_NaNs] = 0
#GLCM_MSI_load = loadtxt('D:/Thesis/Notebooks/Output/Full/GLCM/MSI/GLCM_MSI.csv', delimiter=',')
#img = read_data_EMP(msi)
#nr, nc, nb = img.shape
#nf = 16
#GLCMms = GLCM_MSI_load.reshape(nr,nc,nf)
#where_are_NaNs = isnan(GLCMms)
#GLCMms[where_are_NaNs] = 0
GLCM_BSI_load = loadtxt('D:/Thesis/Notebooks/Output/Full/GLCM/BSI/GLCM_BSI.csv', delimiter=',')
img = read_data_EMP(bsi)
nr, nc, nb = img.shape
nf = 16
GLCMbs = GLCM_BSI_load.reshape(nr,nc,nf)
where_are_NaNs = isnan(GLCMbs)
GLCMbs[where_are_NaNs] = 0

## Run parameter optimalization

parameters to crossreference

IF - N_estimators & Threshold classmap

SVM - Kernel & C & Threshold classmap

In [ ]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(asi)
        stack = np.concatenate((img, EMPas, GLCMas), axis=2)
        stacked = np.dstack((stack, NDVIas, NDWIas))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twita
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30ma
        true2 = Val2GAIAa
        true3 = Val3GHS50a
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score

In [ ]:
#Amsterdam
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('OA' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('P' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('R' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('F' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('AUC' + str(max_key4) + str(max_value4))

In [ ]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(msi)
        stack = np.concatenate((img, EMPms, GLCMms), axis=2)
        stacked = np.dstack((stack, NDVIms, NDWIms))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitm
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mm
        true2 = Val2GAIAm
        true3 = Val3GHS50m
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score

In [ ]:
#Milano
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('OA' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('P' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('R' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('F' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('AUC' + str(max_key4) + str(max_value4))

In [ ]:
%%time
#IF
estimators = [50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]
threshold = [0.3, 0.4, 0.5, 0.6, 0.7]
AOscores = {}
Pscores = {}
Rscores ={}
Fscores = {}
AUCscores = {}


for e in estimators:
    for t in threshold:
        x = str(e+t)
        img = read_data_EMP(bsi)
        stack = np.concatenate((img, EMPbs, GLCMbs), axis=2)
        stacked = np.dstack((stack, NDVIbs, NDWIbs))
        #Setup for IF and OCSVM
        X = stacked
        Ytr1 = Twitb
        nr,nc,nb = X.shape
        ns = nr*nc
        X = X.reshape((ns,nb))
        Ytr = Ytr1.reshape((ns,))
        ind = np.where(Ytr > 0)
        Xtr = X[ind[0],:]
        Ytr = Ytr[ind[0]]
        standard_scaler = StandardScaler()
        Xtr = standard_scaler.fit_transform(Xtr) 
        X = standard_scaler.transform(X)
        model = IsolationForest(n_estimators = e)
        model.fit(Xtr)
        yhat = model.score_samples(X)
        yhat1 = (yhat-min(yhat))/(max(yhat)-min(yhat))
        class_map1 = np.reshape(yhat1,(nr,nc))
        class_map = np.where(class_map1>t, 1, 0)
        class_map = class_map.flatten()
        pred = class_map
        true1 = Val30mb
        true2 = Val2GAIAb
        true3 = Val3GHS50b
        cfm1 = confusion_matrix(true1, pred)
        TN1,FP1,FN1,TP1= cfm1.ravel()
        cfm2 = confusion_matrix(true2, pred)
        TN2,FP2,FN2,TP2= cfm2.ravel()
        cfm3 = confusion_matrix(true3, pred)
        TN3,FP3,FN3,TP3= cfm3.ravel()

        AO = (((TP1+TN1)/(TP1+TN1+FP1+FN1))+
              ((TP2+TN2)/(TP2+TN2+FP2+FN2))+
              ((TP3+TN3)/(TP3+TN3+FP3+FN3)))/3
        AOscores[x] = AO
        
        P = ((TP1/(TP1+FP1))+
             (TP2/(TP2+FP2))+
             (TP3/(TP3+FP3)))/3
        Pscores[x] = P
        
        R = ((TP1/(TP1+FN1))+
             (TP2/(TP2+FN2))+
             (TP3/(TP3+FN3)))/3
        Rscores[x] = R
        
        F = (((2*TP1)/(2*TP1+FP1+FN1))+
             ((2*TP2)/(2*TP2+FP2+FN2))+
             ((2*TP3)/(2*TP3+FP3+FN3)))/3
        Fscores[x] = F
        
        auc1 = roc_auc_score(true1,pred)
        auc2 = roc_auc_score(true2,pred)
        auc3 = roc_auc_score(true3,pred)
        score = ((auc1+auc2+auc3)/3)
        AUCscores[x] = score

In [ ]:
#Budapest
all_values = AOscores.values()
max_value = max(all_values)
max_key = max(AOscores, key=AOscores.get)
print('OA' + str(max_key) + str(max_value))

all_values1 = Pscores.values()
max_value1 = max(all_values1)
max_key1 = max(Pscores, key=Pscores.get)
print('P' + str(max_key1) + str(max_value1))

all_values2 = Rscores.values()
max_value2 = max(all_values2)
max_key2 = max(Rscores, key=Rscores.get)
print('R' + str(max_key2) + str(max_value2))

all_values3 = Fscores.values()
max_value3 = max(all_values3)
max_key3 = max(Fscores, key=Fscores.get)
print('F' + str(max_key3) + str(max_value3))

all_values4 = AUCscores.values()
max_value4 = max(all_values4)
max_key4 = max(AUCscores, key=AUCscores.get)
print('AUC' + str(max_key4) + str(max_value4))